In [ ]:
import argparse

import os
import torch

import matplotlib.pyplot as plt

import numpy as np

import wandb
api = wandb.Api()



from causalode import results_processing_utils



curve_cf_dict = {"random" : [], "uncertainty": []}
curve_f_dict = {"random" : [], "uncertainty": []}
curve_pehe_dict = {"random" : [], "uncertainty": []}

def update_dict(main_dict, new_dict):
    for key in new_dict.keys():
        main_dict[key].append(new_dict[key])

In [ ]:
saving_dir = os.getcwd()
data_path = os.path.join(saving_dir, 'data_created')

data = create_load_save_data(dataset_params, data_path)
cv_data_module = CVDataModule_final(data,  batch_size=args.batch_size, num_workers=9)

cv_data_test_module = CVDataModule_OOD_test(data,  batch_size=args.batch_size, num_workers=9)


In [3]:
checkpoint_path = '/Users/riccardoconci/Local_documents/ACS submissions/THESIS/Counterfactual_ICU/Experiment_6b/dev_gpu_model_checkpoints/best-epoch=68-val_loss=0.00-sd=44-gm=1-cnf=partial-enc=none-encSTD25=False-encSDEd=4-nrmSDE=True-itm=True-txsig=1.0-revert=True-tht=0.01-txmu=0.0-txcw=1-klw=0.001.ckpt.ckpt'

from version_7.model_7 import Hybrid_VAE_SDE

model = Hybrid_VAE_SDE(use_encoder = 'none',
                           start_dec_at_treatment = True, 
                           variational_sampling = False,

                           #Encoder
                           encoder_input_dim =  len(dataset_params['pre_treatment_dims']), 
                           encoder_hidden_dim = args.encoder_hidden_dim,
                           encoder_output_dim = args.expert_latent_dims + args.encoder_SDENN_dims,
                           use_2_5std_encoder_minmax = args.use_2_5std_encoder_minmax,
                           encoder_num_layers = args.encoder_num_layers,
                           variational_encoder = args.variational_encoder,
                           encoder_w_time = args.encoder_w_time,
                           encoder_reverse_time = args.encoder_reverse_time,

                           #SDE params
                           expert_latent_dims  = args.expert_latent_dims,
                           num_samples = args.num_samples,
                           normalise_for_SDENN = args.normalise_for_SDENN,
                           self_reverting_prior_control = args.self_reverting_prior_control,
                           prior_tx_sigma = args.prior_tx_sigma,
                           prior_tx_mu = args.prior_tx_mu,
                           theta = args.theta,
                           SDE_control_weighting = args.SDE_control_weighting,
                           
                           #SDE model params
                           SDE_input_state = args.SDE_input_state, 
                           include_time = args.include_time, 
                           SDEnet_hidden_dim = args.SDEnet_hidden_dim, 
                           SDEnet_depth = args.SDEnet_depth,
                           SDEnet_out_dims = args.SDEnet_out_dims, 
                           final_activation = args.final_activation,

                           #decoder params
                           decoder_output_dims = dataset_params['post_treatment_dims'],
                           normalised_data = dataset_params['normalize'],
                           log_lik_output_scale = args.output_scale,

                           #admin
                           train_dir = os.path.join(saving_dir, 'figures'), 
                           KL_weighting_SDE = args.KL_weighting_SDE, 
                           learning_rate = args.learning_rate,
                           log_wandb = args.log_wandb,
                           adjoint = args.adjoint,
                           plot_every = args.plot_every

    )

model.load_from_checkpoint(checkpoint_path)


model.eval()


TypeError: Hybrid_VAE_SDE.__init__() missing 34 required positional arguments: 'use_encoder', 'start_dec_at_treatment', 'variational_sampling', 'encoder_input_dim', 'encoder_hidden_dim', 'encoder_output_dim', 'encoder_num_layers', 'variational_encoder', 'encoder_w_time', 'encoder_reverse_time', 'use_2_5std_encoder_minmax', 'normalise_for_SDENN', 'expert_latent_dims', 'prior_tx_sigma', 'prior_tx_mu', 'self_reverting_prior_control', 'SDE_input_state', 'include_time', 'theta', 'SDE_control_weighting', 'num_samples', 'SDEnet_hidden_dim', 'SDEnet_depth', 'SDEnet_out_dims', 'final_activation', 'decoder_output_dims', 'log_lik_output_scale', 'normalised_data', 'KL_weighting_SDE', 'train_dir', 'learning_rate', 'log_wandb', 'adjoint', and 'plot_every'

In [ ]:
import wandb
api = wandb.Api()

sweep = api.sweep("edebrouwer/causalode/mrgxzm14") # CV Neural ODE

run_names = [run.id for run in sweep.runs]

repeats = 10
t_lim_trimming_start = 0
t_lim_trimming_end = 10
last_model = True

curve_cf_dict = {"random" : [], "propensity": [], "uncertainty": []}
curve_f_dict = {"random" : [], "propensity": [], "uncertainty": []}
curve_pehe_dict = {"random" : [], "propensity": [], "uncertainty": []}

for run_name in run_names:

    print(run_name)
    PATH = "./wandb/"
    paths = [p for p in os.listdir(PATH) if run_name in p]
    assert(len(paths)==1)
    PATH += paths[0] + "/files/causalode/"+run_name+"/checkpoints/"
    
    if last_model:
        PATH += "last.ckpt"
    else:
        PATH+= [f for f in os.listdir(PATH) if "last" not in f][0]
    model_cls = models.VariationalSDE
    model = model_cls.load_from_checkpoint(PATH)

    hparams = dict(model.hparams)

    if hparams["dataset_name"]=="cv":
        dataset_cls = cv_data_utils.CVDataModule
    elif hparams["dataset_name"]=="covid":
        dataset_cls = covid_data_utils.CovidDataModule
    else:
        dataset_cls = data_utils.PendulumDataModule
    dataset = dataset_cls(**hparams)
    dataset.prepare_data()
    print(f"Loaded model and data from {PATH}")

    model.eval()

    Y_hat_samples_list = []
    Y_alea_std_list = []
    Y_hat_cf_samples_list = []
    Y_alea_std_cf_list = []
    Y_list = []
    Y_cf_list = []
    p_list = []
    T_list = []
    X_list = []

    with torch.no_grad():
        for i,batch in enumerate(dataset.test_dataloader()):
            X,Y, T, Y_cf, p, thetas_0 = batch
            Y_list.append(Y)
            Y_cf_list.append(Y_cf)
            p_list.append(p)
            T_list.append(T)
            X_list.append(X)
            times = torch.arange(11).float()
            #times = torch.arange(11).float()


            Y_hat_list = []
            Y_hat_cf_list = []
            Y_alea_std = []
            Y_alea_cf_std = []
            for _ in range(repeats):
                Y_hat, reg, logqp = model(X,T,return_decoded_u=True, times=times)
                if model.std_dev:
                    Y_hat, Y_std = torch.chunk(Y_hat,2,-1)
                else:
                    #Y_std = model.output_scale.repeat(Y_hat.shape)
                    Y_std = torch.zeros_like(Y_hat)

                Y_hat_list.append(Y_hat)
                Y_alea_std.append(torch.sqrt(torch.sigmoid(Y_std)))

                T_cf = (~T.bool()).float()
                Y_hat_cf, reg, logqp = model(X,T_cf,return_decoded_u=True, times=times)
                if model.std_dev:
                    Y_hat_cf, Y_std_cf = torch.chunk(Y_hat_cf,2,-1)
                else:
                    #Y_std_cf = model.output_scale.repeat(Y_hat_cf.shape)
                    Y_std_cf = torch.zeros_like(Y_hat_cf)
                    
                Y_hat_cf_list.append(Y_hat_cf)
                Y_alea_cf_std.append(torch.sqrt(torch.sigmoid(Y_std_cf)))

            Y_hat_samples = torch.cat(Y_hat_list,0)
            Y_alea_std = torch.cat(Y_alea_std,0)
            Y_hat_cf_samples = torch.cat(Y_hat_cf_list,0)
            Y_alea_std_cf = torch.cat(Y_alea_cf_std,0)

            Y_hat_samples_list.append(Y_hat_samples)
            Y_alea_std_list.append(Y_alea_std)
            Y_hat_cf_samples_list.append(Y_hat_cf_samples)
            Y_alea_std_cf_list.append(Y_alea_std_cf)

    Y_hat_samples = torch.cat(Y_hat_samples_list,1)
    Y_alea_std = torch.cat(Y_alea_std_list,1)
    Y_hat_cf_samples = torch.cat(Y_hat_cf_samples_list,1)
    Y_alea_std_cf = torch.cat(Y_alea_std_cf_list,1)

    Y = torch.cat(Y_list,0)
    Y_cf = torch.cat(Y_cf_list,0)
    p = torch.cat(p_list,0)
    T = torch.cat(T_list,0)
    X = torch.cat(X_list,0)

    Y_hat_mean = Y_hat_samples.mean(0)
    Y_hat_std  = Y_hat_samples.std(0)

    Y_hat_up = (Y_hat_samples+1.96*Y_alea_std).max(0)[0]
    Y_hat_down = (Y_hat_samples-1.96*Y_alea_std).min(0)[0]
    Y_hat_diff = Y_hat_up - Y_hat_down
    
    Y_hat_cf_mean = Y_hat_cf_samples.mean(0)
    Y_hat_cf_std  = Y_hat_cf_samples.std(0)

    Y_hat_cf_up = (Y_hat_cf_samples+1.96*Y_alea_std_cf).max(0)[0]
    Y_hat_cf_down = (Y_hat_cf_samples-1.96*Y_alea_std_cf).min(0)[0]
    Y_hat_cf_diff = Y_hat_cf_up - Y_hat_cf_down

    times_plot = times[1:]
    
    _, cf_dict = results_processing_utils.plot_trimming_prop(Y_cf,Y_hat_cf_mean,Y_hat_cf_diff,t_lim_trimming_start,t_lim_trimming_end,times_plot, data_type = "CFactual", p = (p*T +(1-p)*(1-T)), normalize = False)

    _, f_dict = results_processing_utils.plot_trimming_prop(Y,Y_hat_mean,Y_hat_diff,t_lim_trimming_start,t_lim_trimming_end,times_plot, data_type = "Factual",p = -(p*T +(1-p)*(1-T)), normalize = False)

    _, pehe_dict = results_processing_utils.plot_trimming_prop(Y-Y_cf,Y_hat_mean-Y_hat_cf_mean,Y_hat_diff+Y_hat_cf_diff,t_lim_trimming_start,t_lim_trimming_end,times_plot, data_type = "PEHE", p = torch.abs(p-0.5), normalize = False)

    curve_cf_dict = results_processing_utils.update_dict(curve_cf_dict, cf_dict)
    curve_f_dict = results_processing_utils.update_dict(curve_f_dict, f_dict)
    curve_pehe_dict = results_processing_utils.update_dict(curve_pehe_dict, pehe_dict)